In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import numpy as np
import sys, os, csv

from utils_invitro_global import *

from utils_invitro_misc import parse_noise_filename, find_matching_filename

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib_settings import set_plot_settings, reset_plot_settings

set_plot_settings()

In [ ]:
"""
Noise was measured by shorting the MEA and counter wells through Ag/AgCl electrodes
and taking either 16x16 or 32x32 recording that covers a quarter of the array (centered at array center)
coverage avoided including pixels at the edge of the array because of packaging limitations

each file contains input-referred spectra of recording taken under a particular chip
configuration: PGA gain, HPF cut-off, 256 vs 1024
"""

In [ ]:
load_dir  = f'{REC_DATA_DIR}/noise_sweep'
noise_files = [fn for fn in os.listdir(load_dir) if fn.endswith('.npz')]

Choose Recording Configs

In [ ]:
""" choose which configs to plot: GAIN, HPF """
VGA_GAIN       = 0 # choose from 0, 1, 2, 3.  0: lowest gain. 3: highest gain
VBIAS_PR       = 6 # choose from 5, 6, 7. 5: lower cut-off. 7: higher-cut-off

In [ ]:
""" load data """
noise_fn = find_matching_filename(noise_files, parse_noise_filename,
                                  VGA_GAIN, VBIAS_PR, en_static_elec=1)
tetrode_noise_fn = find_matching_filename(noise_files, parse_noise_filename,
                                  VGA_GAIN, VBIAS_PR, en_static_elec=0)

noise_data = np.load(f'{load_dir}/{noise_fn}')
tetrode_noise_data = np.load(f'{load_dir}/{tetrode_noise_fn}')

In [ ]:
"""
noise data files (npz) have four fields

freq_half: frequency range. dimensions: N
nonsat_ratios: ratio of non-saturated data points for each pixel. dimensions: channel
gain_mu_interp: transfer function used to compute input-referred spectra. output spectra
was divided by this transfer function. dimensions: N
input_fft_powers: input-referred power spectra. dimension: channel*N

N: number of data points
"""

In [ ]:
""" load data """
# 16x16 channel recording data
noise_freqs         = noise_data['freq_half']
noise_nonsat_ratios = noise_data['nonsat_ratios']
noise_Hf            = noise_data['gain_mu_interp']
input_fft_powers    = noise_data['input_fft_powers']

# 32x32 channel recording data
tetrode_noise_freqs         = tetrode_noise_data['freq_half']
tetrode_noise_nonsat_ratios = tetrode_noise_data['nonsat_ratios']
tetrode_noise_Hf            = tetrode_noise_data['gain_mu_interp']
tetrode_input_fft_powers    = tetrode_noise_data['input_fft_powers']

# pixel yield
# noise_nonsat_ratios[noise_nonsat_ratios == 1].sum(), tetrode_noise_nonsat_ratios[tetrode_noise_nonsat_ratios == 1].sum()
nonsat_idx = np.where(noise_nonsat_ratios == 1)[0]
tetrode_nonsat_idx = np.where(tetrode_noise_nonsat_ratios == 1)[0]

In [ ]:
""" Find representative pixels """
# define noise integration bandwidth (Hz)
f_lc, f_hc = 10, 4e3 
fbin, tet_fbin = noise_freqs[1], tetrode_noise_freqs[1] # these two are equal, sans numerical error
i0, i1         = int(f_lc//fbin), int(f_hc//fbin)
tet_i0, tet_i1 = int(f_lc//tet_fbin), int(f_hc//tet_fbin)
assert i0 == tet_i0 and i1 == tet_i1

# compute integrated input-referred noise.
# note the "divide by 2" correction factor. Justification provided at the end of the script.
irns = np.sum(input_fft_powers[nonsat_idx,i0:i1]/2, axis=1)**0.5/1e-6 
tetrode_irns = np.sum(tetrode_input_fft_powers[tetrode_nonsat_idx,tet_i0:tet_i1]/2,
                      axis=1)**0.5/1e-6

# find mean and median
# irn_mean    = np.mean(irns)
irn_median  = np.median(irns)
# tetrode_irn_mean    = np.mean(tetrode_irns)
tetrode_irn_median  = np.median(tetrode_irns)

# find the pixel closest to mean and median
# mean_idx     = np.abs(irns - irn_mean).argmin()
median_idx   = np.abs(irns - irn_median).argmin()
# tetrode_mean_idx     = np.abs(tetrode_irns - tetrode_irn_mean).argmin()
tetrode_median_idx   = np.abs(tetrode_irns - tetrode_irn_median).argmin()

In [ ]:
""" Fetch PSD of a single representative pixels """
valid_input_fft_powers = input_fft_powers[nonsat_idx,:]
valid_tetrode_input_fft_powers = tetrode_input_fft_powers[tetrode_nonsat_idx,:]

# normalize by fbin for plotting
pix_fft_powers = valid_input_fft_powers[median_idx,:]/fbin
tetrode_pix_fft_powers = valid_tetrode_input_fft_powers[tetrode_median_idx,:]/fbin

In [ ]:
""" Plot PSD of a single pixel """
# define plotting range
f_lc, f_hc = 1, 4e3
tol = 1e-2
assert np.abs(noise_freqs[1] - tetrode_noise_freqs[1]) < tol
i0, i1 = int(f_lc//fbin), int(f_hc//fbin)

fig, ax = plt.subplots(figsize=(4, 3))

ax.loglog(noise_freqs[i0:i1], np.sqrt(tetrode_pix_fft_powers[i0:i1]))
ax.loglog(noise_freqs[i0:i1], np.sqrt(pix_fft_powers[i0:i1]))
ax.set_xlim((f_lc, f_hc))
ax.grid(True)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('IRN (V/$\sqrt{Hz}$)')
ax.set_ylim(None, 1e-4)

legend = ax.legend(['32×32 Recording (8.48 kS/s)',
                    '16×16 Recording (33.9 kS/s)'], fontsize=12, loc=(0.015, 0.74))
print(f'Non-tetrode mode IRN: {irns[median_idx]:.2f} Vrms')
print(f'Tetrode mode IRN: {tetrode_irns[tetrode_median_idx]:.2f} Vrms')

In [ ]:
""" Fetch average PSD of XX representative pixels """
# find the pixel closest to mean and median
median_idxs   = np.argsort(np.abs(irns - irn_median))[:10]
tetrode_median_idxs   = np.argsort(np.abs(tetrode_irns - tetrode_irn_median))[:10]

# normalize by fbin for plotting
psd_mean = np.mean(valid_input_fft_powers[median_idxs], axis=0)/fbin
tetrode_psd_mean = np.mean(valid_tetrode_input_fft_powers[median_idxs], axis=0)/fbin

In [ ]:
""" Plot average PSD of XX pixels """
# define plotting range
f_lc, f_hc = 1, 4e3
tol = 1e-2
assert np.abs(noise_freqs[1] - tetrode_noise_freqs[1]) < tol
i0, i1 = int(f_lc//fbin), int(f_hc//fbin)

fig, ax = plt.subplots(figsize=(4, 3))

ax.loglog(noise_freqs[i0:i1], np.sqrt(tetrode_psd_mean[i0:i1]))
ax.loglog(noise_freqs[i0:i1], np.sqrt(psd_mean[i0:i1]))
ax.set_xlim((f_lc, f_hc))
ax.grid(True)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('IRN (V/$\sqrt{Hz}$)')
ax.set_ylim(None, 1e-4)

legend = ax.legend(['32×32 Recording (8.48 kS/s)',
                    '16×16 Recording (33.9 kS/s)'], fontsize=12, loc=(0.015, 0.74))
print(f'Non-tetrode mode IRN: {np.mean(irns[median_idxs]):.2f} Vrms')
print(f'Tetrode mode IRN: {np.mean(tetrode_irns[tetrode_median_idxs]):.2f} Vrms')

In [ ]:
""" Justification on divide by 2 """
# the following was used to compute the power spectra.
def get_fft_power(y, Ts):
    n = len(y)
    freq = np.fft.fftfreq(n, d = Ts)
    half_n = int(np.ceil(n/2.0))
    freq_half = freq[:half_n]

    y_fft = np.fft.fft(y)/n
    # I assume that DC is irrelevant (either not used or zero-meaned prior to taking FFT).
    # Your correction factor will depend on the scaling that is applied here
    y_fft_half = 2*y_fft[:half_n] 
    y_power = np.abs(y_fft_half*np.conj(y_fft_half)) 

    return freq_half, y_power

In [ ]:
""" Test on White Noise Sample """
# sampling rate and duration
sampling_rate = 33900  # 33.9 kHz
duration = 1  # 1 second
num_samples = sampling_rate*duration
# Std Dev of White Noise
noise_std_dev = 1e-3

# Generate a white noise recording
white_noise = np.random.normal(0, noise_std_dev, num_samples)

# Get Spectra
freq_half, noise_power = get_fft_power(white_noise, 1/sampling_rate)

integ_noise = (np.sum(noise_power)/2)**0.5 # Divide by 2 to correct for RMS

""" Verify that the two are equivalent """
print(f"{np.std(white_noise)=:.7f}")
print(f"IRN: {integ_noise:.7f} RMS")